In [ ]:
import dominant_sets
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import random
import matplotlib.cm as cm
import os
from collections import defaultdict, Counter
import math

In [ ]:
# Define the directory containing the dataset
data_directory = r'D:\OneDrive - Delft University of Technology\Thesis\IdiapPosterData_StaticGroupAnnotations(1)\StaticGroupAnnotations'

In [ ]:
# Function to parse each annotation file
def parse_annotation_file(filepath):
    with open(filepath, 'r') as file:
        content = file.readlines()
    
    images_data = []
    for i in range(0, len(content), 5):
        image_info = content[i].split()
        image_path = image_info[-1]  # Last item is the image path
        
        positions_raw = content[i+1].strip()
        positions = positions_raw.replace('(', '').replace(')', '').replace(',', ' ').split()
        positions = [(int(positions[i]), int(positions[i+1])) for i in range(0, len(positions), 2)]
  
        fformations = list(map(int, content[i+2].split()))
        
        images_data.append({
            'image_path': image_path,
            'positions': positions,
            'fformations': fformations
        })
    return images_data

# Aggregate all data from annotators
idiap_annotations = defaultdict(list)
idiap_positions = defaultdict(list)
combined_data = defaultdict(list)
for file_name in os.listdir(data_directory):
    if file_name.endswith('.txt') and file_name != "readme.txt":
        file_path = os.path.join(data_directory, file_name)
        images_data = parse_annotation_file(file_path)
        for image_data in images_data:
            idiap_annotations[image_data['image_path']].append(image_data['fformations'])
            idiap_positions[image_data['image_path']].append(image_data['positions'])
            idiap_positions[image_data['image_path']]         

            # Combine positions and fformations
            grouped_positions = defaultdict(list)
            for pos, label in zip(image_data['positions'], image_data['fformations']):
                grouped_positions[label].append(pos)
            
            # Store the grouped positions in combined_data
            combined_data[image_data['image_path']].append(list(grouped_positions.values()))

In [ ]:
random_pic = "CamA_part1_minipc/camA_part1_003/00000741.jpg"
print(idiap_annotations[random_pic])
print(idiap_positions[random_pic])
print(combined_data[random_pic])
random_pic

In [ ]:
idiap_union_positions = {}

for image_path, positions_lists in idiap_positions.items():
    # Flattening the list of lists and using a set to ensure unique positions
    all_positions = [position for sublist in positions_lists for position in sublist]
    unique_positions = list(set(all_positions))
    idiap_union_positions[image_path] = unique_positions
idiap_union_positions[random_pic]

In [ ]:
# Update combined_data with singletons
updated_combined_data = defaultdict(list)

for image_path, groups_list in combined_data.items():
    all_union_positions = set(idiap_union_positions.get(image_path, []))
    
    for annotator_index, groups in enumerate(groups_list):
        annotated_positions_set = set()
        for group in groups:
            annotated_positions_set.update(group)
        
        singletons = all_union_positions - annotated_positions_set
        
        # Prepare the updated combined groups
        updated_combined_groups = []
        for group in groups:
            updated_combined_groups.append(group)
        
        # Add each singleton as a separate group
        if singletons:
            singleton_groups = [[pos] for pos in singletons]
            updated_combined_groups.extend(singleton_groups)
        
        # Store the updated combined groups for the current annotator
        updated_combined_data[image_path].append(updated_combined_groups)

# updated_combined_data now contains the positions grouped by detected formations and singletons for each image and annotator

In [ ]:
def affinity_calculation(data_dict, sigma = 1.0, method = 'P', dataset = "starfish"):
    affinity_matrices = {}
    valid_members = {}
    
    # if dataset == "starfish":
    #     outer_key = random.choice(list(data_dict.keys()))
    #     inner_dict = data_dict[outer_key]
    #     inner_key = random.choice(list(inner_dict.keys()))
    #     length_of_value = len(inner_dict[inner_key])

    for time, subjects in data_dict.items():
        if dataset == "starfish":
        # Filter out subjects with coordinates [NaN, NaN]
            valid_subjects = {sub_id: coords for sub_id, coords in subjects.items() if not (np.isnan(coords[0]) or np.isnan(coords[1]) or np.isnan(coords[2]))}
            subject_ids = list(valid_subjects.keys())
            coords = np.array(list(valid_subjects.values()))
            positions = coords[:,:2]
            orientations = coords[:,2]
            num_subjects = len(subject_ids)
            valid_members[time] = subject_ids
        elif dataset == "idiap":
            valid_members[time] = subjects
            positions = np.array(subjects)
            num_subjects = len(positions)
            
        else:
            print(f"Doesn't support {dataset}!")
            break
            
        affinity_matrix = np.zeros((num_subjects, num_subjects))
        
        if method == "P":
            for i in range(num_subjects):
                for j in range(num_subjects):
                    # dont set affinity[i][i] to 1, otherwise it would be dominent
                    if i != j:
                        d_ij = np.linalg.norm(positions[i] - positions[j])
                        affinity_matrix[i, j] = np.exp(-d_ij / (2 * sigma**2))
        elif method == "PO":
            for i in range(num_subjects):
                for j in range(num_subjects):
                    if i != j:
                        d_q = np.linalg.norm(positions[i] - positions[j])
                        vector_ij = positions[j] - positions[i]
                        alpha_ij = np.arctan2(vector_ij[1], vector_ij[0])
                        vector_ji = positions[i] - positions[j]
                        alpha_ji = np.arctan2(vector_ji[1], vector_ji[0])
                        
                        theta_i = np.radians(orientations[i])
                        theta_j = np.radians(orientations[j])
                    
                        # Calculate Aori_ij1 and Aori_ij2
                        Aori_ij1 = np.exp(-d_q / (2 * sigma**2)) if -np.pi/2 <= theta_i - alpha_ji <= np.pi/2 else 0
                        Aori_ij2 = np.exp(-d_q / (2 * sigma**2)) if -np.pi/2 <= theta_j - alpha_ij <= np.pi/2 else 0

                        # Final affinity matrix element
                        affinity_matrix[i, j] = min(Aori_ij1, Aori_ij2)
        affinity_matrices[time] = affinity_matrix
    
    return affinity_matrices, valid_members

In [ ]:
import json

In [61]:
# calcualte groups
def extract_fformations(idiap_union_positions, sigma):
    affinities, _ = affinity_calculation(idiap_union_positions, sigma = sigma, method = "P", dataset="idiap")

    random.seed(42)
    selected_time_keys = random.sample(list(affinities.keys()), 10)
    all_time_keys = list(affinities.keys())

    fformations = {}

    for time_key in all_time_keys:
        print('------------------------\n',time_key)
        matrix = affinities[time_key]
        positions = idiap_union_positions[time_key]
        raw_formations = dominant_sets.dominant_set_extraction(matrix, len(affinities[time_key]))
        print(raw_formations)
        
        # Initialize an empty list to hold the groups for this image
        image_groups = []
        detected_members = set()
        
        # Process each detected group in raw_formations
        for group in raw_formations:
            group_positions = [positions[i] for i in range(len(group)) if group[i]]
            image_groups.append(group_positions)
            detected_members.update(i for i, in_group in enumerate(group) if in_group)
        
        # Identify singletons
        # for i, pos in enumerate(positions):
        #     if i not in detected_members:
        #         image_groups.append([pos])
        
        # Store the final f-formations for this image
        fformations[time_key] = image_groups
        
        json_file_path = 'fformations_extracted/'+ str(sigma) + "_fformation.json"
        # print(json_file_path)
        with open(json_file_path, 'w') as json_file:
            json.dump(fformations, json_file)
            
    return fformations

In [62]:
# Function to determine if a detected group matches a ground truth group
def is_group_detected(ground_truth_group, detected_group, threshold):
    intersection = list(set(ground_truth_group) & set(detected_group))
    card_intersection = len(intersection)
    GTcard = len(ground_truth_group)
    groupcard = len(detected_group)
    
    if GTcard == 2 and groupcard == 2:
        if not set(ground_truth_group) ^ set(detected_group):
            return True
    elif card_intersection / max(GTcard, groupcard) >= threshold - math.ulp(1):
        return True
    else:
        return False
    # matched_members = sum(1 for member in detected_group if member in ground_truth_group)
    # required_matches = math.ceil(threshold * len(ground_truth_group))
    
    # unmatched_members = sum(1 for member in detected_group if member not in ground_truth_group)
    # required_unmatches = math.ceil((1-threshold) * len(ground_truth_group))
    # return matched_members >= required_matches and unmatched_members <= required_unmatches
    # return matched_members >= required_matches

# Evaluation function for all images considering multiple annotations
def evaluate_group_detection_multiple_annotations(detected_groups, annotations, threshold=2/3):
    total_tp = 0
    total_fp = 0
    total_fn = 0

    for img_file, detected in detected_groups.items():
        all_annotations = annotations[img_file]
        num_annotations = len(all_annotations)
        image_tp = 0
        image_fp = 0
        image_fn = 0

        # Each annotation set provides a separate ground truth
        for annotation in all_annotations:
            detected_matched = [False] * len(detected)
            ground_truth_matched = [False] * len(annotation)

            for gt_idx, gt_group in enumerate(annotation):
                found = False
                for det_idx, det_group in enumerate(detected):
                    if is_group_detected(gt_group, det_group, threshold):
                        image_tp += 1
                        detected_matched[det_idx] = True
                        ground_truth_matched[gt_idx] = True
                        found = True
                        break
                if not found:
                    image_fn += 1

            image_fp += detected_matched.count(False)

        total_tp += image_tp
        total_fp += image_fp
        total_fn += image_fn

    # Calculate precision, recall, and F1 measure
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_measure = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_measure

In [66]:
for s in range(1, 10):
    print(f"------Sigma: {s}")
    fformations = extract_fformations(idiap_union_positions=idiap_union_positions, sigma=s)
    precision, recall, f1_measure = evaluate_group_detection_multiple_annotations(fformations, combined_data, threshold=2/3)
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-measure: {f1_measure:.2f}")

------Sigma: 1
------------------------
 CamA_part1_minipc/camA_part1_003/00000741.jpg
[array([False,  True, False,  True, False,  True, False, False]), array([False, False, False, False,  True, False, False,  True]), array([ True, False, False, False, False, False,  True, False])]
------------------------
 CamA_part1_minipc/camA_part1_009/00002347.jpg
[array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True]), array([False,  True, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False]), array([False, False, False, False,  True,  True, False, False, False,
       False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False, False,
       False,  True, False,  True, False, False, False]), array([False, False, False, False, False, False, False,  True, False,
        True, False, False, False, False, False, False])]
----